# QC & preprocessing of SlideSeq data

## Libraries & setup

In [1]:
import anndata as ad
import scanpy as sc
import pandas as pd
import os

import numpy as np
from numpy.random import default_rng

import matplotlib.pyplot as plt

sc.logging.print_header()

/nfs/home/dmckellar/mambaforge/envs/sc/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/nfs/home/dmckellar/mambaforge/envs/sc/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/nfs/home/dmckellar/mambaforge/envs/sc/lib/python3.11/site-pac

scanpy==1.9.3 anndata==0.9.2 umap==0.5.3 numpy==1.24.4 scipy==1.11.1 pandas==2.0.3 scikit-learn==1.3.0 statsmodels==0.14.0 python-igraph==0.10.6 pynndescent==0.5.8


/nfs/home/dmckellar/mambaforge/envs/sc/lib/python3.11/site-packages/umap/umap_.py:660: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()


In [2]:
import gc
gc.enable()

In [ ]:
# os.chdir("/path/to/slide_snake") # replace this path with your path...

In [ ]:
from scripts.py.utils import *
from scripts.py.plots import *

## Settings

In [ ]:
# assay = "STAR"
assay = "kb"

print(assay)

Load gene info

In [ ]:
# Read the .tsv file
gtf_info = pd.read_csv("resources/gene_lists/GRCm39_GENCODEM32_ReoT1L_gene_info_gtf.tsv", sep="\t")
gtf_info = gtf_info.drop_duplicates(subset=['GENEID'], keep='first')

# Remove period/decimal suffixes from ensembl IDs
gtf_info['GeneID'] = gtf_info['GENEID'].str.split('.').str[0]

# Extract chromosome value before the colon
gtf_info['chr'] = gtf_info['Chromosome'].str.split(':').str[0]

# Add '.1' to the Reovirus gene names to match the gtf file...
gtf_info["GENEID"] = [f"{GENE}.1" if 'T1LReovirus' in GENE else GENE for GENE in gtf_info['GENEID']]

print(f"Using {len(gtf_info['GENEID'].unique())} ensembl IDs, from {gtf_info['GeneSymbol'].nunique()} genes, and {gtf_info['Biotype'].nunique()} biotypes...\n")

# Display the first few rows
print(gtf_info.head())
# print(gtf_info.tail())

## Load data

In [ ]:
meta = pd.read_csv("resources/metadata_sheets/metadata_Seeker.csv")

In [ ]:
meta = meta.loc[(meta["polyA"] == "none") | (meta["slide_chemistry"] == "dTVN") | ([x in ["SH7", "SH9"] for x in meta["sample"]])]
meta.reset_index(inplace=True, drop=True)
meta

In [ ]:
adata_dict = {}

for i in meta.index:
    if assay == "STAR":
        adata_dict[i] = sc.read_h5ad(
            f"{meta.loc[i, 'data_dir']}/STARsolo/Solo.out/GeneFull/raw/UniqueAndMultEM.h5ad",   
        )
    elif assay == "kb":
        adata_dict[i] = sc.read_h5ad(
            f"{meta.loc[i, 'data_dir']}/kb/raw/output.h5ad"
        )

        # Ensembl IDs -> Gene Names
        convert_feature_names(
            adata_dict[i], 
            gtf_info
        )
        
    print(f"{meta['plot_title'][i]}: {adata_dict[i].shape[0]} spots x {adata_dict[i].shape[1]} genes")
    adata_dict[i].var_names_make_unique()

    adata_dict[i].raw =adata_dict[i]

In [ ]:
tmp_feat = ['Tnp1', "Mir6236"]

sc.set_figure_params(
    dpi=100,
    transparent=True
)

plot_grid_of_embeddings(
    adata_dict,
    # color=['clusters'],
    color=tmp_feat,
    same_scale=False,
    cmap="plasma",
    # size=5,
    vmin=0.00001,
    figsize=(12,3),
    titles=meta['plot_title'],
    # marker='s',
    # use_raw=True,
    basis='spatial'
)

## QC & filtering

In [ ]:
min_counts = 100

for adata in adata_dict.values():
    sc.pp.filter_cells(
        adata, 
        min_counts = min_counts
    )

In [ ]:
for adata in adata_dict.values():
    # Reovirus abundance
    adata.var["Reo"] = adata.var_names.str.startswith("T1LReo")
    sc.pp.calculate_qc_metrics(
        adata, 
        qc_vars=["Reo"], 
        inplace=True
    )

    # Gene biotype %'s
    add_biotypes_pct(
        adata,
        biomart=gtf_info
    )

## Preprocessing

In [ ]:
for adata in adata_dict.values():
    sc.pp.normalize_total(
        adata, 
        target_sum=1e4
    )
    sc.pp.log1p(adata)
    # sc.pp.highly_variable_genes(
    #     adata, 
    #     flavor="seurat", 
    #     n_top_genes=1000
    # )
    sc.pp.pca(adata)
    sc.pp.neighbors(adata)
    sc.tl.leiden(
        adata, 
        key_added="leiden"
    )

In [ ]:
sc.set_figure_params(
    figsize=(10,10),
    dpi=300,
    transparent=True
)

plot_grid_of_embeddings(
    adata_dict,
    color=['leiden'],
    same_scale=False, 
    # plot_titles=meta["plot_title"],
    # size=5,
    cmap='plasma',
    basis='spatial'
)
print(meta["plot_title"])

## Remove clusters containing empty spots

In [ ]:
i=1
sc.pl.embedding(
    adata_dict[i],
    # adata_dict[i][~adata_dict[i].obs['leiden'].isin(['0']), :],
    basis='spatial',
    size=10,
    title=meta['plot_title'][i],
    color='leiden'
)

In [ ]:
for i in meta.index:
    print(f"{meta['plot_title'][i]}: {adata_dict[i].shape[0]} spots x {adata_dict[i].shape[1]} genes")

## Convert spatial coordinates to microns

In [ ]:
i=0
print(f"x range: {min(adata_dict[i].obsm['spatial'][:,0])}, {max(adata_dict[i].obsm['spatial'][:,0])}")
print(f"y range: {min(adata_dict[i].obsm['spatial'][:,1])}, {max(adata_dict[i].obsm['spatial'][:,1])}")

In [ ]:
scale_factor=1/1.44  # from Curio 
print(f"Scale factor: {scale_factor}")

for i in meta.index:
    rescale_embedding(
        adata_dict[i],
        basis='spatial',
        move_to_origin=True,
        scale_factor=scale_factor
    )

## Spatial singlet filtering

Remove spots with fewer than `K` direct neighbors within `D` distance

In [ ]:
for i in meta.index:
    print("====================")
    adata_dict[i] = spatial_singlet_filter(
        adata_dict[i],
        K=2,
        D=45,
        iters=4,
        inplace=False
    )

## Add spliced/unspliced counts as layers

In [ ]:
for i in meta.index:
    for layer in ["spliced", "unspliced"]:
        if assay == "STAR":
            add_mtx_as_layer(
                adata_dict[i],
                mtx_path=f"{meta['data_dir'][i]}/STARsolo/Solo.out/Velocyto/raw/{layer}.mtx.gz",
                obs_path=f"{meta['data_dir'][i]}/STARsolo/Solo.out/Velocyto/raw/barcodes.tsv.gz",
                var_path=f"{meta['data_dir'][i]}/STARsolo/Solo.out/Velocyto/raw/features.tsv.gz",
                layer_name=layer,
                feature_column=1,
                transpose=True,
                verbose=False,
                inplace=True
            )
            adata_dict[i].obs[f'n_counts_{layer}'] = adata_dict[i].layers[layer].sum(axis=1)
            adata_dict[i].obs[f'n_counts_{layer}_log1p'] = np.log1p(adata_dict[i].obs[f'n_counts_{layer}'])
        elif assay == "kb":
            # Temporarily switch feature names to Ensembl IDs 
            adata_dict[i].var_names = adata_dict[i].var['GENEID'] 
            add_mtx_as_layer(
                adata_dict[i],
                mtx_path=f"{meta['data_dir'][i]}/kb_velo/{layer}/output.mtx.gz",
                obs_path=f"{meta['data_dir'][i]}/kb_velo/{layer}/output.barcodes.txt.gz",
                var_path=f"{meta['data_dir'][i]}/kb_velo/{layer}/output.genes.txt.gz",
                layer_name=layer,
                feature_column=0,
                transpose=False,
                verbose=True,
                inplace=True
            )
            adata_dict[i].var_names = adata_dict[i].var['GeneSymbol']
            adata_dict[i].obs[f'n_counts_{layer}'] = adata_dict[i].layers[layer].sum(axis=1)
            adata_dict[i].obs[f'n_counts_{layer}_log1p'] = np.log1p(adata_dict[i].obs[f'n_counts_{layer}'])

In [ ]:
adata_dict[3].obs

In [ ]:
for adata in adata_dict.values():
    adata.obs['splice_ratio'] = adata.obs['n_counts_spliced'] / (adata.obs['n_counts_spliced'] + adata.obs['n_counts_unspliced'])

## Save .h5ad files

In [ ]:
for i in meta.index:
    adata_dict[i].write_h5ad(
        f"data/h5ad/{meta['sample'][i]}_pp.h5ad"
    )

## Plots

In [ ]:
tmp_feat = [
    'log1p_total_counts',
    'n_counts_spliced_log1p',
    # 'splice_ratio',
    'n_counts_unspliced_log1p'
]
sc.set_figure_params(
    dpi=400,
    fontsize=8,
    transparent=True
)
plot_grid_of_embeddings(
    adata_dict,
    # color=['leiden'],
    color=tmp_feat,
    same_scale=True,
    cmap="plasma",
    titles=meta['plot_title'],
    figsize=(10,6),
    colorbar_pad=0,
    # size=5,
    vmin=1,
    hspace=0,
    # use_raw=True,
    basis='spatial'
)

In [ ]:
tmp_feat = [
    'splice_ratio',
    'pct.protein_coding', "pct.rRNA", "pct.miRNA"
]
sc.set_figure_params(
    dpi=400,
    fontsize=8,
    transparent=True
)
plot_grid_of_embeddings(
    adata_dict,
    # color=['leiden'],
    color=tmp_feat,
    same_scale=True,
    cmap="Spectral",
    titles=meta['plot_title'],
    figsize=(10,6),
    colorbar_pad=0,
    # size=5,
    vmin=0.01,
    hspace=0,
    # use_raw=True,
    basis='spatial'
)

In [ ]:
i=3
sc.set_figure_params(
    figsize=(3,3),
    dpi=200,
    fontsize=10,
    transparent=True
)
print(meta['plot_title'][i])
sc.pl.embedding(
    adata_dict[i],
    # adata_dict[i][adata_dict[i].obs['spatial_neighbors_20']<2000],
    # adata_dict[i][-adata_dict[i].obs['leiden'].isin(tmp_cl),:], 
    basis="spatial",
    # size=5**2,
    # title=meta['plot_title'][i],
    vmin=0.0001, vmax=1,
    # vmin=100,
    # vmax=500,
    # color="spatial_neighbors_40"
    color=["leiden","splice_ratio"]
)